In [4]:
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
import os.path
from pathlib import Path
import json
import re

# Loading Model Checking Data

In [9]:
# TODO: all paths extract to global const file
deps_synt_hoa_files = glob("../assets/model-checking-results/synt-with-deps/*.hoa")

def deps_synt_analysis(benchmark_name):
    hoa_file_path = "../assets/model-checking-results/synt-with-deps/{}.hoa".format(benchmark_name)
    json_file_path = "../assets/model-checking-results/synt-with-deps/{}.json".format(benchmark_name)
    hoa_file = Path(hoa_file_path).read_text()
    
    # If JSON file doesn't exist, it we had error in synthesis
    if not os.path.exists(json_file_path):
        status = 'error'
        error = ','.join(re.findall('Runtime error:(.*)', hoa_file))
        
        return {
            'name': benchmark_name,
            'status': 'error',
            'error': error
        }
    
    measures = json.loads(Path(json_file_path).read_text())
    
    duration = measures['total_time']
    status = 'completed' if measures['is_completed'] == True  else 'timeout'
    if status != 'completed':
        return {
            'name': benchmark_name,
            'status': status,
            'duration': duration,
        }
    
    total_dependents = sum((1 for v in measures['tested_variables'] if v['is_dependent']))
    total_outputs = len(measures['output_vars'])
    dependents_ratio = total_dependents / total_outputs
    realizablility = measures['synthesis']['independent_strategy']['realizability']
    model_checking = measures['synthesis']['model_checking_status']
    
    return {
        'name': benchmark_name,
        'status': status,
        'duration': duration,
        'realizablility': realizablility,
        'model_checking': model_checking,
        'dependents_ratio': dependents_ratio,
        'total_dependents': total_dependents
    }

deps_synt_df = pd.DataFrame([
    deps_synt_analysis(Path(benchmark_path).stem)
    for benchmark_path in deps_synt_hoa_files
])

deps_synt_df

,name,status,duration,realizablility,model_checking,dependents_ratio,total_dependents,error
0,TwoCountersInRangeA4,completed,272.0,UNREALIZABLE,UNKNOWN,0.375000,3.0,NaN
1,ltl2dba05,completed,6.0,REALIZABLE,OK,0.000000,0.0,NaN
2,ltl2dba11,completed,17.0,REALIZABLE,OK,0.000000,0.0,NaN
3,TwoCountersDisButA6,completed,8074.0,UNREALIZABLE,UNKNOWN,0.307692,4.0,NaN
4,TorcsSteeringSimple,error,NaN,NaN,NaN,NaN,NaN,Unable to open /tmp/TorcsSteeringSimple.aag
...,...,...,...,...,...,...,...,...
202,OneCounterGuiA1,completed,28.0,UNREALIZABLE,UNKNOWN,0.333333,3.0,NaN
203,EscalatorSmart,error,NaN,NaN,NaN,NaN,NaN,Unable to open /tmp/EscalatorSmart.aag
204,OneCounterGuiA0,completed,49.0,UNREALIZABLE,UNKNOWN,0.333333,3.0,NaN
205,ltl2dba20,completed,36.0,REALIZABLE,OK,0.000000,0.0,NaN


In [8]:
# TODO: all paths extract to global const file
withouts_deps_synt_hoa_files = glob("../assets/model-checking-results/synt-without-deps/*.hoa")

def without_deps_synt_analysis(benchmark_name):
    hoa_file_path = "../assets/model-checking-results/synt-without-deps/{}.hoa".format(benchmark_name)
    json_file_path = "../assets/model-checking-results/synt-without-deps/{}.json".format(benchmark_name)
    hoa_file = Path(hoa_file_path).read_text()
    
    # If JSON file doesn't exist, it we had error in synthesis
    if not os.path.exists(json_file_path):
        status = 'error'
        error = ','.join(re.findall('Runtime error:(.*)', hoa_file))
        
        return {
            'name': benchmark_name,
            'status': 'error',
            'error': error
        }
    
    measures = json.loads(Path(json_file_path).read_text())
    
    duration = measures['total_time']
    status = 'completed' if measures['is_completed'] == True  else 'timeout'
    if status != 'completed':
        return {
            'name': benchmark_name,
            'status': status,
            'duration': duration,
        }
    
    total_dependents = sum((1 for v in measures['tested_variables'] if v['is_dependent']))
    total_outputs = len(measures['output_vars'])
    dependents_ratio = total_dependents / total_outputs
    realizablility = measures['synthesis']['independent_strategy']['realizability']
    model_checking = measures['synthesis']['model_checking_status']
    
    return {
        'name': benchmark_name,
        'status': status,
        'duration': duration,
        'realizablility': realizablility,
        'model_checking': model_checking,
        'dependents_ratio': dependents_ratio,
        'total_dependents': total_dependents
    }

without_deps_synt_df = pd.DataFrame([
    without_deps_synt_analysis(Path(benchmark_path).stem)
    for benchmark_path in withouts_deps_synt_hoa_files
])

without_deps_synt_df

,name,status,duration,realizablility,model_checking,dependents_ratio,total_dependents,error
0,TwoCountersInRangeA4,completed,425.0,UNREALIZABLE,UNKNOWN,0.0,0.0,NaN
1,ltl2dba05,completed,8.0,REALIZABLE,OK,0.0,0.0,NaN
2,ltl2dba11,completed,3.0,REALIZABLE,OK,0.0,0.0,NaN
3,TwoCountersDisButA6,completed,27194.0,UNREALIZABLE,UNKNOWN,0.0,0.0,NaN
4,TorcsSteeringSimple,completed,19.0,REALIZABLE,OK,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
194,OneCounterGuiA1,completed,74.0,UNREALIZABLE,UNKNOWN,0.0,0.0,NaN
195,EscalatorSmart,completed,70.0,REALIZABLE,OK,0.0,0.0,NaN
196,OneCounterGuiA0,completed,43.0,UNREALIZABLE,UNKNOWN,0.0,0.0,NaN
197,ltl2dba20,completed,12.0,REALIZABLE,OK,0.0,0.0,NaN


In [11]:
def seconds_to_ms(secs):
    return secs * 1000

TIMEOUT_MS = seconds_to_ms(60*39) # 39 minutes

# TODO: all paths extract to global const file
spot_sd_hoa_files = glob("../assets/model-checking-results/spot-sd/*.hoa")

def spot_sd_analysis(benchmark_name):
    hoa_file_path = "../assets/model-checking-results/spot-sd/{}.hoa".format(benchmark_name)
    hoa_file = Path(hoa_file_path).read_text()
    
    duration_str = re.findall('real\s+(.*)m(.*)s', hoa_file)
    if len(duration_str) != 1:
        raise Exception(f'Duration not found for benchmark {benchmark_name}, duration = ', duration)

    duration_minutes = float(duration_str[0][0])
    duration_seconds = float(duration_str[0][1])
    duration = seconds_to_ms(duration_seconds + (60*duration_minutes))
    
    is_completed = ('UNREALIZABLE' in hoa_file) or ('REALIZABLE' in hoa_file)
    if not is_completed:
        status = 'timeout' if duration >= TIMEOUT_MS else 'error'
        
        if status == 'error':
            error = ', '.join(re.findall('ltlsynt:(.*)', hoa_file))
        else:
            error = None
        
        return {
            'name': benchmark_name,
            'status': status,
            'error': error,
            'duration': duration
        }
    
    realizablility = 'UNREALIZABLE' if 'UNREALIZABLE' in hoa_file else 'REALIZABLE'
    
    if 'Circuit was verified' in hoa_file:
        model_checking = 'OK'
    elif 'circuit is not OK.' in hoa_file:
        model_checking = 'ERROR'
    else:
        model_checking = 'UNKNOWN'
    
    return {
        'name': benchmark_name,
        'status': 'completed',
        'duration': duration,
        'model_checking': model_checking,
        'realizablility': realizablility
    }

spot_sd_df = pd.DataFrame([
    spot_sd_analysis(Path(benchmark_path).stem)
    for benchmark_path in spot_sd_hoa_files
])

spot_sd_df

,name,status,duration,model_checking,realizablility,error
0,TwoCountersInRangeA4,completed,3161.0,UNKNOWN,UNREALIZABLE,NaN
1,ltl2dba05,completed,69.0,OK,REALIZABLE,NaN
2,ltl2dba11,completed,28.0,OK,REALIZABLE,NaN
3,TorcsSteeringSimple,completed,42.0,OK,REALIZABLE,NaN
4,SensorRegister,completed,41.0,OK,REALIZABLE,NaN
...,...,...,...,...,...,...
195,OneCounterGuiA1,completed,128.0,UNKNOWN,UNREALIZABLE,NaN
196,EscalatorSmart,completed,58.0,OK,REALIZABLE,NaN
197,OneCounterGuiA0,completed,103.0,UNKNOWN,UNREALIZABLE,NaN
198,ltl2dba20,completed,39.0,OK,REALIZABLE,NaN


# Join Model Checking

In [17]:
A = pd.merge(spot_sd_df[["name", "model_checking"]], deps_synt_df[["name", "model_checking"]], on="name", suffixes=('_spot_sd', '_deps_synt'))
model_checking_df = pd.merge(A, without_deps_synt_df[["name", "model_checking"]], on="name")
model_checking_df.rename(columns={"model_checking": "model_checking_without_deps_synt"}, inplace=True)
model_checking_df

,name,model_checking_spot_sd,model_checking_deps_synt,model_checking_without_deps_synt
0,TwoCountersInRangeA4,UNKNOWN,UNKNOWN,UNKNOWN
1,ltl2dba05,OK,OK,OK
2,ltl2dba11,OK,OK,OK
3,TorcsSteeringSimple,OK,NaN,OK
4,SensorRegister,OK,Error,OK
...,...,...,...,...
187,OneCounterGuiA1,UNKNOWN,UNKNOWN,UNKNOWN
188,EscalatorSmart,OK,NaN,OK
189,OneCounterGuiA0,UNKNOWN,UNKNOWN,UNKNOWN
190,ltl2dba20,OK,OK,OK
